# Deloitte's Quantum Climate Challenge 2023

The challenge focuses on CO2 captuere in MOFs (Metal Organic Frameworks) ans is dvided in two principal tasks

## Task 1: Calculate the minimum of the potential energy surface of combinations gas molecules and metallic ions

### Task 1A: Build a quantum/quantum-hybrid algorithm. Run simulations and on real quantum devices

Pick at least one metallic ion from the list:
* Mg2+ (2p6 - 10 e-)
* Mn2+ (3d5 - 23 e-)
* Fe2+ (3d6 - 24 e-)
* Co2+ (3d7 - 25 e-)
* Ni2+ (3d8 - 26 e-)
* Cu2+ (3d9 - 27 e-)
* Zn2+ (3d10 - 28 e-)

And study the composite system with CO2 and another gas molecule:
* CO2 (22 e-)
* H2O (10 e-)
* N2 (14e-)


### Task 1B: Compare those results to classical simulations

For the prupose of this notebook we will focus on Task 1

### Imports

In [140]:
from qiskit.algorithms import VQE, NumPyMinimumEigensolver, NumPyEigensolver #Algorithms

#Qiskit odds and ends
from qiskit.circuit.library import EfficientSU2
from qiskit.algorithms.optimizers import COBYLA, SPSA, SLSQP, L_BFGS_B
from qiskit.opflow import Z2Symmetries, X, Y, Z, I, PauliSumOp, Gradient, NaturalGradient
from qiskit import IBMQ, BasicAer, Aer, transpile
from qiskit.utils import QuantumInstance, algorithm_globals
from qiskit.utils.mitigation import CompleteMeasFitter #Measurement error mitigatioin
from qiskit.tools.visualization import circuit_drawer
from qiskit.providers.aer import AerSimulator
from qiskit.providers.aer.noise import NoiseModel
from qiskit.algorithms.minimum_eigensolvers import AdaptVQE, MinimumEigensolverResult
from qiskit.primitives import Estimator

#qiskit_nature
from qiskit_nature.second_q.drivers import PySCFDriver, MethodType
from qiskit_nature.second_q.formats.molecule_info import MoleculeInfo
from qiskit_nature.units import DistanceUnit
from qiskit_nature.second_q.circuit.library import UCCSD, PUCCD, SUCCD, HartreeFock, CHC, VSCF
from qiskit_nature.second_q.operators.fermionic_op import FermionicOp
from qiskit_nature.second_q.transformers import ActiveSpaceTransformer , FreezeCoreTransformer
from qiskit_nature.second_q.problems import ElectronicStructureProblem, EigenstateResult
from qiskit_nature.second_q.mappers import QubitConverter, ParityMapper, BravyiKitaevMapper, JordanWignerMapper
from qiskit_nature.second_q.algorithms.ground_state_solvers.minimum_eigensolver_factories.vqe_ucc_factory import VQEUCCFactory
from qiskit_nature.second_q.algorithms.ground_state_solvers.minimum_eigensolver_factories.numpy_minimum_eigensolver_factory import NumPyMinimumEigensolverFactory
from qiskit_nature.second_q.algorithms.ground_state_solvers import GroundStateEigensolver
from qiskit_nature.second_q.algorithms.excited_states_solvers.eigensolver_factories.numpy_eigensolver_factory import NumPyEigensolverFactory
from qiskit_nature.second_q.algorithms.excited_states_solvers import QEOM, ExcitedStatesEigensolver

#Runtime
from qiskit_ibm_runtime import (QiskitRuntimeService, Session,
                                Estimator as RuntimeEstimator)
from qiskit_ibm_runtime.options import Options, ResilienceOptions, SimulatorOptions, TranspilationOptions, ExecutionOptions

#PySCF
from functools import reduce
import scipy.linalg
from pyscf import scf
from pyscf import gto, dft
from pyscf import mcscf, fci
from functools import reduce
from pyscf.mcscf import avas, dmet_cas

#Python odds and ends
import matplotlib
import matplotlib.pyplot as plt
import matplotlib
import pylab
import numpy as np
import os
import pyscf
from IPython.display import display, clear_output
import mapomatic as mm

from datetime import datetime
%matplotlib inline
%config InlineBackend.figure_format = 'svg' # Makes the images look nice

IBMQ.load_account()
provider = IBMQ.get_provider(group='deployed')
service = QiskitRuntimeService(channel='ibm_quantum')
#set Backends
#simulators
backend_stv = Aer.get_backend('aer_simulator_statevector')
#Real Devices
backend_nair= provider.get_backend('ibm_nairobi')
backend_manil = provider.get_backend('ibmq_manila')
backend_qsm_ibm=provider.get_backend('ibmq_qasm_simulator')

#solvers
npme = NumPyMinimumEigensolver()
npe=NumPyEigensolver()

ibmqfactory.load_account:WARNING:2023-01-18 17:08:21,952: Credentials are already in use. The existing account in the session will be replaced.


In [2]:
## Python program to store list to file using pickle module
import pickle

# write list to binary file
def write_list(a_list,filename):
    # store list in binary file so 'wb' mode
    with open(filename, 'wb') as fp:
        pickle.dump(a_list, fp)
        print('Done writing list into a binary file')
        
def write_dict(a_dict,filename):
    # store list in binary file so 'wb' mode
    with open(filename, 'wb') as fp:
        pickle.dump(a_dict, fp,protocol=pickle.HIGHEST_PROTOCOL)
        print('Done writing dict into a binary file')

# Read list to memory
def read(filename):
    # for reading also binary mode is important
    with open(filename, 'rb') as fp:
        n_list = pickle.load(fp)
        return n_list

In [3]:
#Create a custoom VQE Program
from qiskit.algorithms.minimum_eigensolvers import MinimumEigensolver, VQEResult

class CustomVQE(MinimumEigensolver):
    
    def __init__(self, estimator, circuit, optimizer, callback=None):
        self._estimator = estimator
        self._circuit = circuit
        self._optimizer = optimizer
        self._callback = callback
        self._eval_count=0
        
    def compute_minimum_eigenvalue(self, operators,initial_point, aux_operators=None):
                
        # Define objective function to classically minimize over
        def objective(x):
            # Execute job with estimator primitive
            job = self._estimator.run([self._circuit], [operators], [x])
            # Get results from jobs
            est_result = job.result()
            # Get the measured energy value
            value = est_result.values[0]
            std=est_result.metadata[0]
            self._eval_count+=1
            # Save result information using callback function
            if self._callback is not None:
                self._callback(self._eval_count, value, std)
            return value    
        # Select an initial point for the ansatzs' parameters
        x0 =initial_point
        #x0=0*np.random.rand(self._circuit.num_parameters)
        
        # Run optimization
        res = self._optimizer.minimize(objective, x0=x0)
        
        # Populate VQE result
        result = VQEResult()
        result.cost_function_evals = res.nfev
        result.eigenvalue = res.fun
        result.optimal_parameters = res.x
        return result

counts = []
values = []
deviation = []
def callback(eval_count, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts.append(eval_count)
    values.append(mean)
    deviation.append(std)
    
counts_sim = []
values_sim = []
deviation_sim = []
def callback_sim(eval_count, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts_sim.append(eval_count)
    values_sim.append(mean)
    deviation_sim.append(std)
    
counts_real = []
values_real = []
deviation_real = []
def callback_real(eval_count, mean, std):  
    # Overwrites the same line when printing
    display("Evaluation: {}, Energy: {}, Std: {}".format(eval_count, mean, std))
    clear_output(wait=True)
    counts_real.append(eval_count)
    values_real.append(mean)
    deviation_real.append(std)


In [228]:
def exact_solver(problem, converter):
    solver = NumPyMinimumEigensolverFactory()
    calc = GroundStateEigensolver(converter, solver)
    result = calc.solve(problem)
    return result

def local_solver(distances,mapper,optimizer,freeze_core):
    
    dists=[]
    results=[]
    problems=[]
    for dist in distances:
        #Driver
        driver,og_problem=make_driver(dist)
        #Qubit_Op
        qubit_op, problem, converter = make_qubit_op(og_problem,mapper,freeze_core)
        #Initial State
        init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, converter)
        #ansatz = UCCSD(num_spatial_orbitals=problem.num_spatial_orbitals,num_particles=problem.num_particles,qubit_converter=converter,initial_state=init_state)
        ansatz = EfficientSU2(num_qubits=qubit_op.num_qubits,su2_gates=['ry'], entanglement='linear', reps=3, initial_state=init_state)
        # Set initial parameters of the ansatz
        initial_point = np.pi/4 * np.random.rand(ansatz.num_parameters)
        
        estimator = Estimator([ansatz], [qubit_op])
    
        counts = []
        values = []
        deviation = []
    
        custom_vqe = CustomVQE(estimator, ansatz, optimizer, callback=callback)

        result = custom_vqe.compute_minimum_eigenvalue(qubit_op,initial_point=initial_point)
        
        results.append(result)
        dists.append(dist)
        problems.append(problem)
    
    return results, problems ,distances

def sim_solver(distances, mapper, optimizer,freeze_core,est_options,mapomatic,device):
    
    dists=[]
    results=[]
    problems=[]
                             
    for dist in distances:
        #Driver
        driver,og_problem=make_driver(dist)
        #Qubit_Op
        qubit_op,problem,converter = make_qubit_op(og_problem,mapper,freeze_core=freeze_core)
        #Initial State
        init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, converter)
        #ansatz = UCCSD(num_spatial_orbitals,num_particles,converter)
        ansatz = EfficientSU2(num_qubits=qubit_op.num_qubits,su2_gates='ry', entanglement='linear', reps=3, initial_state=init_state)
        if mapomatic==True:
            ansatz_opt = transpile(ansatz, backend=provider.get_backend(device),optimization_level=3,routing_method='sabre')
            small_qc = mm.deflate_circuit(ansatz_opt)
            layouts = mm.matching_layouts(small_qc, provider.get_backend(device))
            scores = mm.evaluate_layouts(small_qc, layouts, provider.get_backend(device))
            ansatz = transpile(small_qc, backend=provider.get_backend(device),initial_layout=scores[0][0],optimization_level=3,routing_method='sabre')
        # Set initial parameters of the ansatz
        initial_point = np.pi/4 * np.random.rand(ansatz.num_parameters)
    
        count_sim = []
        values_sim = []
        deviation_sim = []
    
        with Session(service=service, backend='ibmq_qasm_simulator') as session:
            # Prepare primitive
            rt_estimator = RuntimeEstimator(session=session,options=est_options)
            # Set up algorithm
            custom_vqe = CustomVQE(rt_estimator, ansatz, optimizer, callback=callback_sim)
            # Run algorithm
            result= custom_vqe.compute_minimum_eigenvalue(qubit_op,initial_point)

        results.append(result)      
            
        dists.append(dist)
        problems.append(problem)
       
        
    return results,problems, distances

def real_solver(distances, mapper, optimizer,freeze_core,est_options, device):
    
    dists=[]
    results=[]
    problems=[]
                             
    for dist in distances:
        #Driver
        driver,og_problem=make_driver(dist)
        #Qubit_Op
        qubit_op, num_particles, num_spatial_orbitals, converter, problem = make_qubit_op(og_problem,mapper,freeze_core=freeze_core)
        #Initial State
        init_state = HartreeFock(problem.num_spatial_orbitals, problem.num_particles, converter)
        #ansatz = UCCSD(num_spatial_orbitals,num_particles,converter)
        ansatz = EfficientSU2(num_qubits=qubit_op.num_qubits,su2_gates='ry', entanglement='linear', reps=3, initial_state=init_state)
        #ansatz_opt = transpile(ansatz, backend=provider.get_backend(device),optimization_level=3,routing_method='sabre')
        #small_qc = mm.deflate_circuit(ansatz_opt)
        #layouts = mm.matching_layouts(small_qc, provider.get_backend(device))
        #scores = mm.evaluate_layouts(small_qc, layouts, provider.get_backend(device))
        #ansatz = transpile(small_qc, backend=provider.get_backend(device),initial_layout=scores[0][0],optimization_level=3,routing_method='sabre')
        
        # Set initial parameters of the ansatz
        initial_point = np.pi/4 * np.random.rand(ansatz.num_parameters)
    
        counts_real = []
        values_real = []
        deviation_real = []
    
        with Session(service=service, backend=device) as session:
            # Prepare primitive
            rt_estimator = RuntimeEstimator(session=session,options=est_options)
            # Set up algorithm
            custom_vqe = CustomVQE(rt_estimator, ansatz, optimizer, callback=callback_sim)
            # Run algorithm
            result = custom_vqe.compute_minimum_eigenvalue(qubit_op,initial_point)

        results.append(result)
        
            
        dists.append(dist)
        problems.append(problem)
       
        
    return results,problems, distances

def classic_solver(distances,mapper,optimizer,freeze_core):
    
    results=[]
    dists=[]
    for dist in distances:
        #Driver
        driver,og_problem=make_driver(dist)
        #Qubit_Op
        qubit_op, problem,converter = make_qubit_op(og_problem,mapper,freeze_core=freeze_core)
        result= exact_solver(problem, converter = converter)
        results.append(result)
        dists.append(dist)
    return results,dists

# Calculations

Let's start with smallest system first

## Mg2+ + H2O

In [115]:
#Define Molecule
d=1.5
molecule = MoleculeInfo(
             # coordinates in Angstrom
                     symbols=['O','H','H','Mg'],
                     coords=[
                            (d+0.504284,0.0,0.758602),
                            (d,0.0,0.0),
                            (d+2*0.504284,0.0,0.0),
                            (0.0, 0.0, 0.0),
                            ],
                     multiplicity=1,  # = 2*spin + 1
                     charge=2,
                     units=DistanceUnit.ANGSTROM
                    )

#Set driver
driver = PySCFDriver.from_molecule(molecule, basis="sto3g", method=MethodType.RKS)

#Get properties
og_problem = driver.run()
    
#particle_number = problem.grouped_property_transformed.get_property("ParticleNumber")
#num_a_electrons = problem.num_alpha
#num_b_electrons = propblem.num_beta
og_num_particles=og_problem.num_particles

og_num_spatial_orbitals =og_problem.num_spatial_orbitals

og_rep_energy = og_problem.nuclear_repulsion_energy
    
# Check the occupation of the spin orbitals
og_PN_property = og_problem.properties.particle_number
mol = gto.Mole()
mol.atom = [
        ['O',(d+0.504284,0.0,0.758602)],
        ['H',(d,0.0,0.0),],
        ['H',(d+2*0.504284,0.0,0.0)],
        ['Mg',(0.0, 0.0, 0.0)]
        ]
mol.charge=2
mol.basis = 'sto3g'
mol.spin = 0
mol.build()

mf = scf.RKS(mol).x2c()
mf.kernel()

ao_labels = ['Mg 2p', 'O 2p','H 2s']
avas_obj = avas.AVAS(mf, ao_labels)
avas_obj.kernel()
weights=np.append(avas_obj.occ_weights,avas_obj.vir_weights)
weights=(weights>0.1)*weights
orbs=np.nonzero(weights)
# Define the active space
# (selected automatically around the HOMO and LUMO, ordered by energy)
transformer = ActiveSpaceTransformer(
            num_electrons=(int(avas_obj.nelecas/2),int(avas_obj.nelecas/2)), #Electrons in active space
            num_spatial_orbitals=avas_obj.ncas, #Orbitals in active space
            active_orbitals=orbs[0].tolist()
        )

fz_transformer=FreezeCoreTransformer(freeze_core=True)
    
#Define the problem

problem=transformer.transform(og_problem)
fz_problem=fz_transformer.transform(problem)

hamiltonian=problem.hamiltonian
second_q_op = hamiltonian.second_q_op()
fz_hamiltonian=fz_problem.hamiltonian
fz_second_q_op = fz_hamiltonian.second_q_op() 
    
num_spatial_orbitals = problem.num_spatial_orbitals
num_particles = problem.num_particles

fz_num_spatial_orbitals = fz_problem.num_spatial_orbitals
fz_num_particles = fz_problem.num_particles
    
mapper=ParityMapper()

#Get Pauli OP
#converter = QubitConverter(mapper)
#qubit_op = converter.convert(hamiltonian)
    
#Symmetries
#sym=Z2Symmetries.find_Z2_symmetries(qubit_op)

converter = QubitConverter(mapper,two_qubit_reduction=True, z2symmetry_reduction='auto')
simple_converter = QubitConverter(mapper)
    
#Final OP
qubit_op = converter.convert(second_q_op,num_particles=num_particles,sector_locator=problem.symmetry_sector_locator)
fz_qubit_op = simple_converter.convert(fz_second_q_op)

converged SCF energy = -270.842130093854


In [186]:
problem.num_particles[0]

6

In [117]:
og_weights

array([-1.98693787e-16, -9.66581511e-17,  1.70534284e-16,  1.86061819e-16,
        7.71005053e-01,  8.96644610e-01,  9.69819683e-01,  9.94498816e-01,
        9.94520238e-01,  9.94761777e-01,  2.12202549e-01, -1.81386222e-17,
        4.66093028e-04,  5.30447683e-04,  6.80697545e-03,  8.59773309e-02])

In [101]:
orbs[0].tolist()

[4, 5, 6, 7, 8, 9, 15]

In [92]:
qubit_op.num_qubits

11

In [93]:
fz_qubit_op.num_qubits

2

In [209]:
def make_driver(d):
    
    
    molecule = MoleculeInfo(
             # coordinates in Angstrom
                     symbols=['O','H','H','Mg'],
                     coords=[
                            (d+0.504284,0.0,0.758602),
                            (d,0.0,0.0),
                            (d+2*0.504284,0.0,0.0),
                            (0.0, 0.0, 0.0),
                            ],
                     multiplicity=1,  # = 2*spin + 1
                     charge=2,
                     units=DistanceUnit.ANGSTROM
                    )
    
    #Set driver
    driver = PySCFDriver.from_molecule(molecule, basis="sto3g", method=MethodType.RKS)

    #Get properties
    problem = driver.run()
    

    return driver, problem

def make_qubit_op(og_problem, mapper,freeze_core):
    mol = gto.Mole()
    mol.atom = [
        ['O',(d+0.504284,0.0,0.758602)],
        ['H',(d,0.0,0.0),],
        ['H',(d+2*0.504284,0.0,0.0)],
        ['Mg',(0.0, 0.0, 0.0)]
        ]
    mol.charge=2
    mol.basis = 'sto3g'
    mol.spin = 0
    mol.build()

    mf = scf.RKS(mol).x2c()
    mf.kernel()

    ao_labels = ['Mg 2p', 'O 2p','H 2s']
    avas_obj = avas.AVAS(mf, ao_labels)
    avas_obj.kernel()
    weights=np.append(avas_obj.occ_weights,avas_obj.vir_weights)
    weights=(weights>0.1)*weights
    orbs=np.nonzero(weights)
    orbs=np.nonzero(weights)
    
    transformer = ActiveSpaceTransformer(
            num_electrons=(int(avas_obj.nelecas/2),int(avas_obj.nelecas/2)), #Electrons in active space
            num_spatial_orbitals=avas_obj.ncas, #Orbitals in active space
            active_orbitals=orbs[0].tolist()
        )
    fz_transformer=FreezeCoreTransformer(freeze_core=freeze_core)
    
    #Define the problem

    problem=transformer.transform(og_problem)
    if freeze_core==True:
        problem=fz_transformer.transform(problem)
        converter = QubitConverter(mapper)
    else:
        converter = QubitConverter(mapper,two_qubit_reduction=True, z2symmetry_reduction='auto')

    hamiltonian=problem.hamiltonian
    second_q_op = hamiltonian.second_q_op()
    
    num_spatial_orbitals = problem.num_spatial_orbitals
    num_particles = problem.num_particles
    
    qubit_op = converter.convert(second_q_op,num_particles=num_particles,sector_locator=problem.symmetry_sector_locator)
        

    
    return qubit_op, problem,  converter
    

In [162]:
from qiskit.providers.fake_provider import FakeManila, FakeNairobi, FakeMontreal, FakeGuadalupe

#Noisy simulator
backend_aer = AerSimulator(method='density_matrix',device='CPU')
device_backend=FakeGuadalupe()
device = AerSimulator.from_backend(device_backend)
noise_model = None
coupling_map = device.configuration().coupling_map
noise_model = NoiseModel.from_backend(device)
basis_gates = noise_model.basis_gates

sim_opt=SimulatorOptions(
    noise_model=noise_model,
    seed_simulator=62,
    coupling_map=coupling_map,
    basis_gates=basis_gates
)

In [164]:
#Runtime Estimator options
ts_opt=TranspilationOptions(
    skip_transpilation=True
)

sim_opt=SimulatorOptions(
    noise_model=noise_model,
    seed_simulator=62,
    coupling_map=coupling_map,
    basis_gates=basis_gates
)
res_opt=ResilienceOptions(
    noise_factors=tuple(range(1, 6, 2)),
    noise_amplifier='LocalFoldingAmplifier',
    extrapolator='LinearExtrapolator'
)

ex_opt=ExecutionOptions(
    shots=1024
)
est_options=Options(
    resilience_level=2,
    #optimization_level=3,
    execution=ex_opt,
    resilience=res_opt,
    simulator=sim_opt,
    #transpilation=ts_opt
)

In [226]:
distances = np.arange(0.5, 0.7, 0.1)
optimizer = SLSQP(maxiter=50)
mapper=ParityMapper()


seed = 62
algorithm_globals.random_seed = seed

In [227]:
counts = []
values = []
deviation = []

results,problems,dists=local_solver(                                               distances=distances,
                                                                          mapper=mapper,
                                                                          optimizer=optimizer,freeze_core=False)

KeyboardInterrupt: 

In [222]:
print(problems[0].interpret(results[0]))

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -334.025532307999
  - computed part:      -62.398833972488
  - ActiveSpaceTransformer extracted energy part: -271.626698335511
~ Nuclear repulsion energy (Hartree): 67.092350119878
> Total ground state energy (Hartree): -266.933182188122
 
=== MEASURED OBSERVABLES ===
 
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [18.97821711  0.0  11.46840014]
 


In [166]:
results,problems,dists=sim_solver(distances=distances,
                             mapper=mapper,
                             optimizer=optimizer,
                             freeze_core=False,
                             est_options=est_options,
                             mapomatic=False,
                             device='ibmq_kolkata'    
                            )

converged SCF energy = -270.842130093854


KeyboardInterrupt: 

In [223]:
results,dists=classic_solver(distances=distances,
                             mapper=mapper,
                             optimizer=optimizer,
                             freeze_core=False,
                            )

converged SCF energy = -270.842130093854
converged SCF energy = -270.842130093854


In [225]:
print(results[0])

=== GROUND STATE ENERGY ===
 
* Electronic ground state energy (Hartree): -336.342521622078
  - computed part:      -64.715823286567
  - ActiveSpaceTransformer extracted energy part: -271.626698335511
~ Nuclear repulsion energy (Hartree): 67.092350119878
> Total ground state energy (Hartree): -269.2501715022
 
=== MEASURED OBSERVABLES ===
 
  0:  # Particles: 12.000 S: 0.000 S^2: 0.000 M: 0.000
 
=== DIPOLE MOMENTS ===
 
~ Nuclear dipole moment (a.u.): [18.97821711  0.0  11.46840014]
 
  0: 
  * Electronic dipole moment (a.u.): [18.81488981  None  12.6322088]
    - computed part:      [14.78344957  None  9.78433507]
    - ActiveSpaceTransformer extracted energy part: [4.03144025  0.0  2.84787373]
  > Dipole moment (a.u.): [0.1633273  None  -1.16380866]  Total: None
                 (debye): [0.41513655  None  -2.95810628]  Total: None
 


In [51]:
orbs

NameError: name 'orbs' is not defined